In [1]:
import pandas as pd
import geopandas as gdp

In [5]:
# Import accidents data
data = pd.read_excel("../Data/Segment_Cluster_ID.xls")
data.head()

,OBJECTID_1,Join_Count,TARGET_FID,OBJECTID,Cod_Uniq_F,Cod_Uniq,Name_Hoze,Cod_Hoze,Name_Mehva,Y,...,Onvan_File,Date_1,Seg_name,Long_,Lat,ID,Unique_ID,Seg_Cluster_id,id_1,V1
0,1,1,1,168,,H3-29R,حوزه نيشابور,,نیشابور-سبزوار,1.368700e+06,...,,1997.06.19,59.0,58.778848,36.237378,42,59-42,59-0,1.0,0.0
1,2,1,2,171,,H3-43R,حوزه نيشابور,,نیشابور-سبزوار,1.366845e+06,...,,1997.06.28,58.0,58.739444,36.221944,42,58-42,58-0,2.0,0.0
2,3,1,3,179,,H3-29R,حوزه نيشابور,,نیشابور-سبزوار,1.368701e+06,...,,1997.08.13,59.0,58.779169,36.237341,44,59-44,59-0,3.0,0.0
3,4,1,4,183,,H3-36R,حوزه نيشابور,,نیشابور-سبزوار,1.367938e+06,...,,1997.07.15,59.0,58.758239,36.231234,20,59-20,59-1,4.0,1.0
4,5,1,5,186,,H3-32R,حوزه نيشابور,,نیشابور-سبزوار,1.368537e+06,...,,1997.07.14,59.0,58.769115,36.236176,30,59-30,59-2,5.0,2.0


In [6]:
# Drop unneeded columns
data = data.drop(columns=data.columns[1:116])
data.head()

,OBJECTID_1,Seg_name,Long_,Lat,ID,Unique_ID,Seg_Cluster_id,id_1,V1
0,1,59.0,58.778848,36.237378,42,59-42,59-0,1.0,0.0
1,2,58.0,58.739444,36.221944,42,58-42,58-0,2.0,0.0
2,3,59.0,58.779169,36.237341,44,59-44,59-0,3.0,0.0
3,4,59.0,58.758239,36.231234,20,59-20,59-1,4.0,1.0
4,5,59.0,58.769115,36.236176,30,59-30,59-2,5.0,2.0


In [4]:
data["Long_"] == data.Long_

0      True
1      True
2      True
3      True
4      True
       ... 
875    True
876    True
877    True
878    True
879    True
Name: Long_, Length: 880, dtype: bool

In [7]:
# Create geodataframe from accidents data
geodata = gdp.GeoDataFrame(data, geometry=gdp.points_from_xy(data.Long_, data.Lat))
geodata.head()

,OBJECTID_1,Seg_name,Long_,Lat,ID,Unique_ID,Seg_Cluster_id,id_1,V1,geometry
0,1,59.0,58.778848,36.237378,42,59-42,59-0,1.0,0.0,POINT (58.77885 36.23738)
1,2,58.0,58.739444,36.221944,42,58-42,58-0,2.0,0.0,POINT (58.73944 36.22194)
2,3,59.0,58.779169,36.237341,44,59-44,59-0,3.0,0.0,POINT (58.77917 36.23734)
3,4,59.0,58.758239,36.231234,20,59-20,59-1,4.0,1.0,POINT (58.75824 36.23123)
4,5,59.0,58.769115,36.236176,30,59-30,59-2,5.0,2.0,POINT (58.76912 36.23618)


In [8]:
# Group geodataframe rows by the corresponding Segment-ID and Cluster-ID
index_list = []
max_dist = []
for name, group in geodata.groupby(["Seg_name", "V1"]):
    max_dist.append(group["geometry"].apply(lambda point: group["geometry"].distance(point)).max().max() * 100000)
    index_list.append(name)

In [9]:
# Create a dataframe from maximum distance, cluster-ID, and segment-ID of each cluster
Final = pd.DataFrame(list(zip(index_list, max_dist)), columns =['Seg_Cluster', 'Max_Dist'])

In [10]:
Final["Seg_id"] = Final["Seg_Cluster"].apply(lambda x: x[0])
Final["Cluster_id"] = Final["Seg_Cluster"].apply(lambda x: x[1])
Final.head()

,Seg_Cluster,Max_Dist,Seg_id,Cluster_id
0,"(1.0, -1.0)",2068.244346,1.0,-1.0
1,"(1.0, 0.0)",565.669476,1.0,0.0
2,"(1.0, 1.0)",166.641138,1.0,1.0
3,"(2.0, -1.0)",4012.330357,2.0,-1.0
4,"(2.0, 0.0)",254.168173,2.0,0.0


In [140]:
# Save results to excel file
Final.to_excel("Final.xlsx")